In [1]:
import os
os.chdir("/app")

In [2]:
import pandas as pd
from pathlib import Path
from nbgrader.api import Gradebook
from nbgrader.apps import NbGraderAPI
from traitlets.config import Config
from tqdm.auto import tqdm
import re
import os
import time

In [3]:
COURSE_ID = "shad_1_2025_august"

In [4]:
config = Config()
config.CourseDirectory.course_id = COURSE_ID
# config.ExecutePreprocessor.environment = {
#     'DATA_PATH_2': '/path/to/data'
# }
config.ExecutePreprocessor.timeout = 3600

In [5]:
nbg = NbGraderAPI(config=config)
gradebook = nbg.gradebook

In [6]:
ASSIGNMENT = "hw_1"
NOTEBOOK = "hse_metrics"  # without .ipynb
# MAX_SCORE = 20

# Collect

In [7]:
nbg.collect(ASSIGNMENT, update=False)

{'success': True,
 'log': "[INFO] Processing 36 submissions of 'hw_1' for course 'shad_1_2025_august'\n[WARNING] \n    /usr/local/share/nbgrader/exchange/shad_1_2025_august/inbound/Ясакова_Анастасия+hw_1+2025-08-31T21:00:02 UTC+00 claims to be submitted by Ясакова_Анастасия but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Collecting submission: Ясакова_Анастасия hw_1\n[WARNING] \n    /usr/local/share/nbgrader/exchange/shad_1_2025_august/inbound/Пудовкин_Никита+hw_1+2025-08-31T21:00:05 UTC+00 claims to be submitted by Пудовкин_Никита but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Collecting submission: Пудовкин_Никита hw_1\n[WARNING] \n    /usr/local/share/nbgrader/exchange/shad_1_2025_august/inbound/Василенко_Егор+hw_1+2025-08-31T21:03:25 UTC+00 claims to be submitted by Василенко_Егор but is owned by dev; cheating a

# Autograde

In [8]:
submitted_student_ids = nbg.get_submitted_students(ASSIGNMENT)
type(submitted_student_ids), len(submitted_student_ids)

(set, 36)

In [9]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
type(autograded_student_ids), len(autograded_student_ids)

(set, 0)

In [10]:
not_autograded_student_ids = list(submitted_student_ids - autograded_student_ids)
len(not_autograded_student_ids)

36

In [11]:
student_ids_requested_to_grade = None

In [12]:
if student_ids_requested_to_grade is not None:
    student_ids_to_grade = student_ids_requested_to_grade
    force = True
else:
    student_ids_to_grade = not_autograded_student_ids
    force = False

In [13]:
f"{force=}", student_ids_to_grade

('force=False',
 ['Асатуров_Максим',
  'Можеитов_Матвей',
  'Шумский_Игорь',
  'Сухарев_Денис',
  'Галаева_Екатерина',
  'Проходцев_Егор',
  'Брызгалов_Владимир',
  'Гришкин_Иван',
  'Рукавица_Артём',
  'Данилкина_Дарья',
  'Коренев_Артем',
  'Футьянов_Михаил',
  'Белоглазова_Анастасия',
  'Гагарина_Ульяна',
  'Медведько_Тимофей',
  'Карюкина_Ксения',
  'Ромадова_Ирина',
  'Пестрякова_Елизавета',
  'Рышкова_София',
  'Смирнов_Владимир',
  'Мерберг_Вероника',
  'Василенко_Егор',
  'Бромберг_Анастасия',
  'Пироженко_Анастасия',
  'Шестакова_Анна',
  'Тюлякова_Ольга',
  'Селивёрстов_Никита',
  'Ясакова_Анастасия',
  'Аминов_Рустам',
  'Украинцева_Елена',
  'Суханов_Григорий',
  'Андреева_Ангелина',
  'Таскаев_Владислав',
  'Соков_Владислав',
  'Пудовкин_Никита',
  'Галкин_Артемий'])

In [21]:
student_id = student_ids_to_grade[0]
force = False

In [22]:
res = nbg.autograde(assignment_id=ASSIGNMENT, student_id=student_id, force=force, create=True)

In [23]:
res

{'success': False,
 'error': 'Traceback (most recent call last):\n  File "/usr/local/lib/python3.10/site-packages/nbgrader/utils.py", line 542, in capture_log\n    app.start()\n  File "/usr/local/lib/python3.10/site-packages/nbgrader/converters/base.py", line 132, in start\n    self.convert_notebooks()\n  File "/usr/local/lib/python3.10/site-packages/nbgrader/converters/base.py", line 468, in convert_notebooks\n    raise NbGraderException(msg)\nnbgrader.converters.base.NbGraderException: Please see the the above traceback for details on the specific errors on the above failures.\n',
 'log': '[INFO] Creating/updating student with ID \'Асатуров_Максим\': {}\n[INFO] Overwriting files with master versions from the source directory\n[INFO] Sanitizing /app/submitted/Асатуров_Максим/hw_1/hse_metrics.ipynb\n[INFO] Converting notebook /app/submitted/Асатуров_Максим/hw_1/hse_metrics.ipynb\n[WARNING] Attribute \'checksum\' for cell cell-c6bdee220c984461 has changed! (should be: f64c6135b05bb43eaf

In [17]:
results = {}
for student_id in tqdm(student_ids_to_grade):
    ts = time.time()
    res = nbg.autograde(assignment_id=ASSIGNMENT, student_id=student_id, force=force, create=True)
    elapsed = time.time() - ts
    res["time"] = elapsed
    results[student_id] = res

  0%|          | 0/36 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/nbgrader/api.py:2255: SAWarning: Object of type <SubmittedAssignment> not in session, add operation along 'Student.submissions' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happened while initializing objects.)
  for notebook in submission.assignment.notebooks:
/usr/local/lib/python3.10/site-packages/nbgrader/api.py:2255: SAWarning: Object of type <SubmittedAssignment> not in session, add operation along 'Assignment.submissions' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happened while initializing objects.)
  for notebook in submission.assignment.notebooks:
/usr/local/lib/python3.10/site-packages/nbgra

In [18]:
successes = {sid: res for sid, res in results.items() if res["success"]}
failures = {sid: res for sid, res in results.items() if not res["success"]}
len(successes), len(failures)

(0, 36)

In [19]:
failures

{'Асатуров_Максим': {'success': False,
  'error': 'Traceback (most recent call last):\n  File "/usr/local/lib/python3.10/site-packages/nbgrader/utils.py", line 542, in capture_log\n    app.start()\n  File "/usr/local/lib/python3.10/site-packages/nbgrader/converters/base.py", line 132, in start\n    self.convert_notebooks()\n  File "/usr/local/lib/python3.10/site-packages/nbgrader/converters/base.py", line 468, in convert_notebooks\n    raise NbGraderException(msg)\nnbgrader.converters.base.NbGraderException: Please see the the above traceback for details on the specific errors on the above failures.\n',
  'log': '[INFO] Creating/updating student with ID \'Асатуров_Максим\': {}\n[INFO] Overwriting files with master versions from the source directory\n[INFO] Sanitizing /app/submitted/Асатуров_Максим/hw_1/hse_metrics.ipynb\n[INFO] Converting notebook /app/submitted/Асатуров_Максим/hw_1/hse_metrics.ipynb\n[WARNING] Attribute \'checksum\' for cell cell-c6bdee220c984461 has changed! (should 

# Get per-task scores

In [24]:
student_ids_requested_to_grade = None

In [28]:
if student_ids_requested_to_grade is not None:
    students_to_get_scores = student_ids_requested_to_grade
else:
    students_to_get_scores = nbg.get_autograded_students(ASSIGNMENT)

In [30]:
len(students_to_get_scores)

36

In [7]:
students_to_get_scores = ["Орлов_Александр"]

In [8]:
scores = {}
for student_id in tqdm(students_to_get_scores):
    nb = gradebook.find_submission_notebook(NOTEBOOK, ASSIGNMENT, student_id)
    student_scores = {
        "total": {"actual": nb.score, "max": nb.max_score},
        "cells": [
            {"actual": g.score, "max": g.max_score}
            for g in nb.grades
        ]
    }
    scores[student_id] = student_scores

  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
total_scores = {sid: int(s["total"]["actual"]) for sid, s in scores.items()}
len(total_scores)

1

In [10]:
total_scores

{'Орлов_Александр': 8}

In [11]:
scores

{'Орлов_Александр': {'total': {'actual': 8.0, 'max': 20.0},
  'cells': [{'actual': 1.0, 'max': 1.0},
   {'actual': 1.0, 'max': 1.0},
   {'actual': 1.0, 'max': 1.0},
   {'actual': 1.0, 'max': 1.0},
   {'actual': 1.0, 'max': 1.0},
   {'actual': 1.0, 'max': 1.0},
   {'actual': 1.0, 'max': 1.0},
   {'actual': 1.0, 'max': 1.0},
   {'actual': 0.0, 'max': 6.0},
   {'actual': 0.0, 'max': 6.0}]}}

# Make human-readable feedback

In [12]:
CELL_NAMES = [
    "precision_at_1",  
    "precision_at_5", 
    "recall_at_1", 
    "ap_at_3_for_user_2", 
    "map_at_3",
    "dcg_at_3_for_user_2",
    "idcg_at_3_for_user_2",
    "ndcg_at_3",
    "weighted_recall_correctness",
    "weighted_recall_effectiveness",
]

**Скоры предполагаются целыми**

In [13]:
feedbacks = []
for student_id, student_scores in scores.items():
    total_score = student_scores["total"]
    feedback_scores = [f"total: {total_score['actual']:.0f} / {total_score['max']:.0f}"]
    for cell_name, cell_score in zip(CELL_NAMES, student_scores["cells"]):
        feedback_scores.append(f"{cell_name}: {cell_score['actual']:.0f} / {cell_score['max']:.0f}")
    feedback = {
        "student_id": student_id,
        "feedback": ", ".join(feedback_scores),
        "score": round(total_score["actual"])
    }
    feedbacks.append(feedback)

In [14]:
feedbacks

[{'student_id': 'Орлов_Александр',
  'feedback': 'total: 8 / 20, precision_at_1: 1 / 1, precision_at_5: 1 / 1, recall_at_1: 1 / 1, ap_at_3_for_user_2: 1 / 1, map_at_3: 1 / 1, dcg_at_3_for_user_2: 1 / 1, idcg_at_3_for_user_2: 1 / 1, ndcg_at_3: 1 / 1, weighted_recall_correctness: 0 / 6, weighted_recall_effectiveness: 0 / 6',
  'score': 8}]

In [37]:
df = pd.DataFrame(feedbacks)
df

,student_id,feedback,score
0,Асатуров_Максим,"total: 14 / 20, precision_at_1: 1 / 1, precisi...",14
1,Можеитов_Матвей,"total: 14 / 20, precision_at_1: 1 / 1, precisi...",14
2,Шумский_Игорь,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20
3,Сухарев_Денис,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20
4,Галаева_Екатерина,"total: 8 / 20, precision_at_1: 1 / 1, precisio...",8
5,Проходцев_Егор,"total: 15 / 20, precision_at_1: 0 / 1, precisi...",15
6,Брызгалов_Владимир,"total: 15 / 20, precision_at_1: 0 / 1, precisi...",15
7,Гришкин_Иван,"total: 14 / 20, precision_at_1: 0 / 1, precisi...",14
8,Рукавица_Артём,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20
9,Данилкина_Дарья,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20


In [38]:
df.to_csv("hw_1_results.csv")